## Suppose the Google Sycamore gate is the 2-qubit gate in your gate-set. How many Sycamore gates would it take to implement a CCNOT gate?

In [125]:
import qiskit
import numpy as np

We will start by defining the Sycamore gate and CCNOT gate:
Sycamore:


|   1     0     0    0   |

|   0     0    -i    0   |

|   0    -i     0    0   |

|   0     0     0    e^(-i*pi/6)|

CCNOT:

|1 0 0 0 0 0 0 0|

|0 1 0 0 0 0 0 0|

|0 0 1 0 0 0 0 0|

|0 0 0 1 0 0 0 0|

|0 0 0 0 1 0 0 0|

|0 0 0 0 0 1 0 0|

|0 0 0 0 0 0 0 1|

|0 0 0 0 0 0 1 0|

So our objective is to create a circuit that starts with the Sycamore gate, and uses that (as well as other 1 qubit gates) to end up with the CCNOT gate. To do this, we need to start by creating the Sycamore gate in qiskit:

In [81]:

backend = qiskit.Aer.get_backend('unitary_simulator')

circ = qiskit.QuantumCircuit(2)
circ.cx(1,0)
circ.cx(0,1)
circ.p((3*np.pi)/2, 0)
circ.cx(1,0)
circ.cp((-1*np.pi)/6, 0 ,1)

sycamore = circ.to_instruction(None, "Sycamore")


print(circ)

job = qiskit.execute(circ, backend, shots=8192)
result = job.result()

print(result.get_unitary(circ,3))

     ┌───┐     ┌─────────┐┌───┐          
q_0: ┤ X ├──■──┤ P(3π/2) ├┤ X ├─■────────
     └─┬─┘┌─┴─┐└─────────┘└─┬─┘ │P(-π/6) 
q_1: ──■──┤ X ├─────────────■───■────────
          └───┘                          
Operator([[ 1.   +0.j ,  0.   +0.j ,  0.   +0.j ,  0.   +0.j ],
          [ 0.   -0.j ,  0.   -0.j , -0.   -1.j ,  0.   -0.j ],
          [ 0.   -0.j , -0.   -1.j ,  0.   -0.j ,  0.   -0.j ],
          [ 0.   +0.j ,  0.   +0.j ,  0.   +0.j ,  0.866-0.5j]],
         input_dims=(2, 2), output_dims=(2, 2))


As you can see from the matrix above, we have successfully created the sycamore gate. Now, we need to use the Sycamore gate to create a more familiar 2-qubit gate, the CNOT gate, so that we can use it implement a CCNOT gate:

In [118]:
cnot = qiskit.QuantumCircuit(2)
cnot.h(0)
cnot.append(sycamore, [0,1])
cnot.append(sycamore, [0,1])
cnot.append(sycamore, [0,1])
cnot.append(sycamore, [0,1])
cnot.append(sycamore, [0,1])
cnot.append(sycamore, [0,1])
cnot.z([0,1])
cnot.h(0)

syc_cnot = cnot.to_instruction(None, "SCN")

print(cnot)

job = qiskit.execute(cnot, backend, shots=8192)
result = job.result()

print(result.get_unitary(cnot,3))

     ┌───┐┌───────────┐┌───────────┐┌───────────┐┌───────────┐┌───────────┐»
q_0: ┤ H ├┤0          ├┤0          ├┤0          ├┤0          ├┤0          ├»
     └───┘│  Sycamore ││  Sycamore ││  Sycamore ││  Sycamore ││  Sycamore │»
q_1: ─────┤1          ├┤1          ├┤1          ├┤1          ├┤1          ├»
          └───────────┘└───────────┘└───────────┘└───────────┘└───────────┘»
«     ┌───────────┐┌───┐┌───┐
«q_0: ┤0          ├┤ Z ├┤ H ├
«     │  Sycamore │├───┤└───┘
«q_1: ┤1          ├┤ Z ├─────
«     └───────────┘└───┘     
Operator([[ 1.-0.j,  0.+0.j,  0.+0.j,  0.+0.j],
          [ 0.+0.j,  1.-0.j,  0.+0.j,  0.+0.j],
          [-0.+0.j, -0.+0.j,  0.-0.j,  1.-0.j],
          [-0.+0.j, -0.+0.j,  1.-0.j,  0.-0.j]],
         input_dims=(2, 2), output_dims=(2, 2))


We can now see that 6 sycamore gates can be used to create a CNOT gate. Finally, we can now use these 6 sycamore gate CNOT's to create a CCNOT gate, following the CCNOT decomposition found here: https://www.semanticscholar.org/paper/On-the-CNOT-cost-of-TOFFOLI-gates-Shende-Markov/c7011bd2c3eb2dbd4e8879ffb9803dc221c17c09

In [124]:
ccnot = qiskit.QuantumCircuit(3)
ccnot.h(0)
ccnot.append(syc_cnot, [0,1])
ccnot.tdg(0)
ccnot.append(syc_cnot, [0,2])
ccnot.t(0)
ccnot.append(syc_cnot, [0,1])
ccnot.tdg(0)
ccnot.barrier()
ccnot.append(syc_cnot, [0,2])
ccnot.t([1,0])
ccnot.h(0)
ccnot.append(syc_cnot, [1,2])
ccnot.tdg(1)
ccnot.t(2)
ccnot.append(syc_cnot, [1,2])

print(ccnot)

job = qiskit.execute(ccnot, backend, shots=8192)
result = job.result()

print(result.get_unitary(ccnot,3))

     ┌───┐┌──────┐┌─────┐┌──────┐┌───┐┌──────┐┌─────┐ ░ ┌──────┐┌───┐ ┌───┐  »
q_0: ┤ H ├┤0     ├┤ Tdg ├┤0     ├┤ T ├┤0     ├┤ Tdg ├─░─┤0     ├┤ T ├─┤ H ├──»
     └───┘│  SCN │└─────┘│      │└───┘│  SCN │└─────┘ ░ │      │├───┤┌┴───┴─┐»
q_1: ─────┤1     ├───────┤  SCN ├─────┤1     ├────────░─┤  SCN ├┤ T ├┤0     ├»
          └──────┘       │      │     └──────┘        ░ │      │└───┘│  SCN │»
q_2: ────────────────────┤1     ├─────────────────────░─┤1     ├─────┤1     ├»
                         └──────┘                     ░ └──────┘     └──────┘»
«                    
«q_0: ───────────────
«     ┌─────┐┌──────┐
«q_1: ┤ Tdg ├┤0     ├
«     └┬───┬┘│  SCN │
«q_2: ─┤ T ├─┤1     ├
«      └───┘ └──────┘
Operator([[ 1.-0.j, -0.-0.j, -0.+0.j, -0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
            0.+0.j],
          [ 0.-0.j,  1.-0.j,  0.-0.j, -0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
            0.+0.j],
          [ 0.+0.j, -0.-0.j,  1.-0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
            0.+0.j],
        

## From the paper and circuit above, we can see that it is possible to decompose a CCNOT gate using 6 CNOT gates, bringing our total Sycamore gate count to 36.